# 开发外星人游戏，综合第十二到第十四章

In [1]:
import sys
import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### 注意其中self.settings = Settings()与self.ship = Ship(self)的区别，加self是因为还要调用主程序中的函数

In [4]:
import sys
import pygame

class AlienInvasion:
    
    def __init__(self):
        # 初始化游戏
        pygame.init()
        
        # 控制帧率的方法
        self.clock = pygame.time.Clock()
        
        self.settings = Settings()
        
        self.screen = pygame.display.set_mode((self.settings.screen_width, self.settings.screen_height)) 

        pygame.display.set_caption("Alien Invasion")
        
        self.ship = Ship(self)
        
        self.bullets = pygame.sprite.Group()
    
    # 开始游戏的主循环
    def run_game(self):
        """开始主游戏循环"""
        while True:
            self._check_events()
            self.ship.update()
            self._update_bullets()      
            self._update_screen()           
            self.clock.tick(60)
    
    # 相应按键与鼠标事件
    def _check_events(self):
        # 监听鼠标与键盘
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    sys.exit()
                elif event.type == pygame.KEYDOWN: 
                    self._check_keydown_events(event) 
                elif event.type == pygame.KEYUP: 
                    self._check_keyup_events(event) 
    
    # 进行重构
    def _check_keydown_events(self, event): 
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = True
        elif event.key == pygame.K_LEFT: 
            self.ship.moving_left = True
        elif event.key == pygame.K_q: 
            sys.exit() 
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
    
    def _check_keyup_events(self, event): 
        if event.key == pygame.K_RIGHT: 
            self.ship.moving_right = False 
        elif event.key == pygame.K_LEFT: 
            self.ship.moving_left = False
            
    def _fire_bullet(self):
        """创建子弹并且开火"""
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)
            
    def _update_bullets(self):
        """更新子弹位置并且删除已经消失的子弹"""
        # 更新位置
        self.bullets.update() 
            
        # 删除已经消失的子弹
        for bullet in self.bullets.copy(): 
            if bullet.rect.bottom <= 0:
                self.bullets.remove(bullet) 
            
    def _update_screen(self):
        # 更新屏幕上的图像，并切换到新屏幕
        self.screen.fill(self.settings.bg_color)
        for bullet in self.bullets.sprites(): 
            bullet.draw_bullet()
        self.ship.blitme() 

        # 让最近绘制的屏幕可见
        pygame.display.flip()
    
if __name__ == 'main':
    # 创建游戏实例并运行
    ai = AlienInvasion()
    ai.run_game()

In [5]:
ai = AlienInvasion()
ai.run_game()

SystemExit: 

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


创建Setting类，便于添加新的功能

In [3]:
class Settings: 
    """存储游戏《外星⼈⼊侵》中所有设置的类""" 
    def __init__(self): 
 
        self.screen_width = 1200 
        self.screen_height = 800 
        self.bg_color = (230, 230, 230)
        self.ship_speed = 1.5
        
        # 加入子弹设置
        self.bullet_speed = 2
        self.bullet_width = 3
        self.bullet_height = 15 
        self.bullet_color = (60, 60, 60)
        self.bullets_allowed = 3

开始绘制飞船

In [2]:
import pygame

class Ship: 
    def __init__(self, ai_game):
        self.screen = ai_game.screen 
        self.screen_rect = ai_game.screen.get_rect()
        
        # 加载⻜船图像并获取其外接矩形
        self.image = pygame.image.load('image.png')
        self.rect = self.image.get_rect()
        
        # 每艘新⻜船都放在屏幕底部的中央
        self.rect.midbottom = self.screen_rect.midbottom
        
        # 在⻜船的属性x 中存储⼀个浮点数
        self.x = float(self.rect.x)
        
        # 移动标志（⻜船⼀开始不移动）
        self.moving_right = False
        self.moving_left = False 
        
        self.settings = ai_game.settings
        
    def update(self): 
        # 更新⻜船⽽不是rect对象的 x 值
        if self.moving_right and self.rect.right < self.screen_rect.right:
            self.x += self.settings.ship_speed 
        if self.moving_left and self.rect.left > 0:
            self.x -= self.settings.ship_speed 
         # 根据self.x 更新rect对象
        self.rect.x = self.x 
        
        # 根据移动标志调整⻜船的位置
        if self.moving_right: 
            self.rect.x += self.settings.ship_speed 
        if self.moving_left: 
            self.rect.x -= self.settings.ship_speed
        
        # 根据self.x 更新rect 的值对象
        self.rect.x = self.x 

    def blitme(self):
        # 在指定位置绘制⻜船
        self.screen.blit(self.image, self.rect)

创建bullet类

In [1]:
import pygame 
from pygame.sprite import Sprite

class Bullet(Sprite):
    """管理飞船所有发射的子弹类"""
    
    def __init__(self, ai_game):
        """在飞船当前位置创建子弹"""
        super().__init__()
        self.screen = ai_game.screen 
        self.settings = ai_game.settings 
        self.color = self.settings.bullet_color
        
        # 在(0,0)处创建⼀个表⽰⼦弹的矩形，再设置正确的位置
        self.rect = pygame.Rect(0, 0, self.settings.bullet_width,self.settings.bullet_height)
        self.rect.midtop = ai_game.ship.rect.midtop
        
        # 存储⽤浮点数表⽰的⼦弹位置
        self.y = float(self.rect.y)
        
    def update(self):
        """更新子弹的位置"""
        self.y -= self.settings.bullet_speed
        # 更新表⽰⼦弹的 rect 的位置
        self.rect.y = self.y
        
    def draw_bullet(self):
        """在屏幕上绘制⼦弹"""
        pygame.draw.rect(self.screen, self.color, self.rect)

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
